In [225]:
# เริ่มรับข้อมูล
# Lastname, prefix.Name
import pandas as pd
input_data = ['Klaew, Mr. Chaiwat']
data = input_data
X_test = pd.DataFrame(data, columns= ['Name'])
X = X_test.Name
df_text = X

In [226]:
### 1. ตัดคำแยกเป็น : LastName, Prefix. name ###
# กรณีถ้า คอลมน์ไม่ถึง 4 ให้สร้าง index มาเติมให้ครบ 4

In [227]:
df_text

0    Klaew, Mr. Chaiwat
Name: Name, dtype: object

In [228]:
from nltk.tokenize import WhitespaceTokenizer

tokenized_lists = [] # สร้าง list ไว้รอเก็บ เครื่องมือตัดคำ, ค่าตำแหน่ง
for i in range(0,len(X)):
    tokenized_lists.append((WhitespaceTokenizer(), i)) # เก็บ เครื่องมือตัดคำ, ค่าตำแหน่ง(ไว้ใช้ต่อไป)
tokenized_lists

[(WhitespaceTokenizer(pattern='\\s+', gaps=True, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>),
  0)]

In [229]:
    
results = []
for tokenized, i in tokenized_lists:
    tokenized_list = tokenized.tokenize(X[i]) 
    results.append(tokenized_list) # เอาผลไว้ใน results

In [230]:
results

[['Klaew,', 'Mr.', 'Chaiwat']]

In [231]:
### 2. แยก column เป็น : LastName,Prefix_name. Name
import pandas as pd    
# เก็บข้อมูลลง DataFrame
df_text = pd.DataFrame(results)  # ยังไม่ต้องระบุชื่อคอลัมน์
# การตัดคำจะถูกแยกออกเป็นตัดแล้วได้ 1.ได้คำที่คอลัมน์ 1 พอดี 2. ไปอยู่ที่คอลัมน์ 2 แก้โดยหา idx_fail แล้วเปลี่ยน ณ index นั้นๆ

# กำจัด missing data ด้วยการแทน str : ' '
for i in range(len(df_text.columns)):
    df_text[i] = df_text[i].fillna(' ')

    

In [232]:
# กรณีถ้า คอลมน์ไม่ถึง 4 ให้สร้าง index มาเติมให้ครบ 4
df_text.head()
#for i in range(4):
x = len(df_text.columns)
final = 5
for x in range(x,4):
    print(x)
    df_text[x] = ''

3


In [233]:
df_text.head()

,0,1,2,3
0,"Klaew,",Mr.,Chaiwat,


In [234]:
Prefix_list = ['Mr.', 'Mrs.', 'Miss.', 'Master.', 'Don.', 'Rev.', 'Dr.', 'Mme.', 'Ms.', 'Major.', 'Mlle.', 'Col.', 'Capt.', 'Jonkheer.', 'Sir.', 'Lady.', 'Dona.']
Prefix_special = ''
    
idx_fail = df_text[1].loc[
        
                          (df_text[1] != Prefix_list[0]) & (df_text[1]!= Prefix_list[1]) & (df_text[1] !=Prefix_list[2]) &  (df_text[1] !=Prefix_list[3]) &
                          (df_text[1] != Prefix_list[4]) & (df_text[1]!= Prefix_list[5]) & (df_text[1] !=Prefix_list[6]) &  (df_text[1] !=Prefix_list[7]) &
                          (df_text[1] != Prefix_list[8]) & (df_text[1]!= Prefix_list[9]) & (df_text[1] !=Prefix_list[10]) &  (df_text[1] !=Prefix_list[11]) &
                          (df_text[1] != Prefix_list[12]) & (df_text[1]!= Prefix_list[13]) & (df_text[1] !=Prefix_list[14]) & (df_text[1] !=Prefix_list[15])
        
                          ] 

In [235]:
len(df_text.columns)            

4

In [236]:
# โค้ด python function สำหรับนับพยางค์
import re
def syllables(word) :
    word = word.lower()

    # exception_add are words that need extra syllables
    # exception_del are words that need less syllables

    exception_add = ['serious','crucial']
    exception_del = ['fortunately','unfortunately']

    co_one = ['cool','coach','coat','coal','count','coin','coarse','coup','coif','cook','coign','coiffe','coof','court']
    co_two = ['coapt','coed','coinci']

    pre_one = ['preach']

    syls = 0 #added syllable number
    disc = 0 #discarded syllable number

    #1) if letters < 3 : return 1
    if len(word) <= 3 :
        syls = 1
        return syls

    #2) if doesn't end with "ted" or "tes" or "ses" or "ied" or "ies", discard "es" and "ed" at the end.
    # if it has only 1 vowel or 1 set of consecutive vowels, discard. (like "speed", "fled" etc.)

    if word[-2:] == "es" or word[-2:] == "ed" :
        doubleAndtripple_1 = len(re.findall(r'[eaoui][eaoui]',word))
        if doubleAndtripple_1 > 1 or len(re.findall(r'[eaoui][^eaoui]',word)) > 1 :
            if word[-3:] == "ted" or word[-3:] == "tes" or word[-3:] == "ses" or word[-3:] == "ied" or word[-3:] == "ies" :
                pass
            else :
                disc+=1

    #3) discard trailing "e", except where ending is "le"  

    le_except = ['whole','mobile','pole','male','female','hale','pale','tale','sale','aisle','whale','while']

    if word[-1:] == "e" :
        if word[-2:] == "le" and word not in le_except :
            pass

        else :
            disc+=1

    #4) check if consecutive vowels exists, triplets or pairs, count them as one.

    doubleAndtripple = len(re.findall(r'[eaoui][eaoui]',word))
    tripple = len(re.findall(r'[eaoui][eaoui][eaoui]',word))
    disc+=doubleAndtripple + tripple

    #5) count remaining vowels in word.
    numVowels = len(re.findall(r'[eaoui]',word))

    #6) add one if starts with "mc"
    if word[:2] == "mc" :
        syls+=1

    #7) add one if ends with "y" but is not surrouned by vowel
    if word[-1:] == "y" and word[-2] not in "aeoui" :
        syls +=1

    #8) add one if "y" is surrounded by non-vowels and is not in the last word.

    for i,j in enumerate(word) :
        if j == "y" :
            if (i != 0) and (i != len(word)-1) :
                if word[i-1] not in "aeoui" and word[i+1] not in "aeoui" :
                    syls+=1

    #9) if starts with "tri-" or "bi-" and is followed by a vowel, add one.

    if word[:3] == "tri" and word[3] in "aeoui" :
        syls+=1

    if word[:2] == "bi" and word[2] in "aeoui" :
        syls+=1

    #10) if ends with "-ian", should be counted as two syllables, except for "-tian" and "-cian"

    if word[-3:] == "ian" : 
    #and (word[-4:] != "cian" or word[-4:] != "tian") :
        if word[-4:] == "cian" or word[-4:] == "tian" :
            pass
        else :
            syls+=1

    #11) if starts with "co-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:2] == "co" and word[2] in 'eaoui' :

        if word[:4] in co_two or word[:5] in co_two or word[:6] in co_two :
            syls+=1
        elif word[:4] in co_one or word[:5] in co_one or word[:6] in co_one :
            pass
        else :
            syls+=1

    #12) if starts with "pre-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:3] == "pre" and word[3] in 'eaoui' :
        if word[:6] in pre_one :
            pass
        else :
            syls+=1

    #13) check for "-n't" and cross match with dictionary to add syllable.

    negative = ["doesn't", "isn't", "shouldn't", "couldn't","wouldn't"]

    if word[-3:] == "n't" :
        if word in negative :
            syls+=1
        else :
            pass   

    #14) Handling the exceptional words.

    if word in exception_del :
        disc+=1

    if word in exception_add :
        syls+=1     

    # calculate the output
    return numVowels - disc + syls

In [237]:
# import library
from nltk.tokenize import WhitespaceTokenizer
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler as Scaler # Normalization 0-1 เพราะ Variance มีค่าต่างกันแต่ละตัวแปร

def preprocessing(X):
    
    ### 1. ตัดคำแยกเป็น : LastName, Prefix. name ###
    
    tokenized_lists = [] # สร้าง list ไว้รอเก็บ เครื่องมือตัดคำ, ค่าตำแหน่ง
    for i in range(len(X)):
        tokenized_lists.append((WhitespaceTokenizer(), i)) # เก็บ เครื่องมือตัดคำ, ค่าตำแหน่ง(ไว้ใช้ต่อไป)    
    results = []
    for tokenized, i in tokenized_lists:
        tokenized_list = tokenized.tokenize(X[i]) 
        results.append(tokenized_list) # เอาผลไว้ใน results
        
    ### 2. แยก column เป็น : last_name,prefix_name, name ###
    
    # เก็บข้อมูลลง DataFrame
    df_text = pd.DataFrame(results)  # ยังไม่ต้องระบุชื่อคอลัมน์
    # การตัดคำจะถูกแยกออกเป็นตัดแล้วได้ 1.ได้คำที่คอลัมน์ 1 พอดี 2. ไปอยู่ที่คอลัมน์ 2 แก้โดยหา idx_fail แล้วเปลี่ยน ณ index นั้นๆ
    # กำจัด missing data ด้วยการแทน str : ' '
    for i in range(len(df_text.columns)):
        df_text[i] = df_text[i].fillna(' ')
        
        
    # กรณีถ้า คอลัมน์ไม่ถึง 4 ให้สร้าง index มาเติมให้ครบ 4
    df_text.head()
    x = len(df_text.columns) # จำนวนคอลัมน์ df ที่มี
    final = 5
    for x in range(x,4):
        df_text[x] = ''
        
    Prefix_list = ['Mr.', 'Mrs.', 'Miss.', 'Master.', 'Don.', 'Rev.', 'Dr.', 'Mme.', 'Ms.', 'Major.', 'Mlle.', 'Col.', 'Capt.', 'Jonkheer.', 'Sir.', 'Lady.', 'Dona.']
    
    idx_fail = df_text[1].loc[
        
                          (df_text[1] != Prefix_list[0]) & (df_text[1]!= Prefix_list[1]) & (df_text[1] !=Prefix_list[2]) &  (df_text[1] !=Prefix_list[3]) &
                          (df_text[1] != Prefix_list[4]) & (df_text[1]!= Prefix_list[5]) & (df_text[1] !=Prefix_list[6]) &  (df_text[1] !=Prefix_list[7]) &
                          (df_text[1] != Prefix_list[8]) & (df_text[1]!= Prefix_list[9]) & (df_text[1] !=Prefix_list[10]) &  (df_text[1] !=Prefix_list[11]) &
                          (df_text[1] != Prefix_list[12]) & (df_text[1]!= Prefix_list[13]) & (df_text[1] !=Prefix_list[14]) & (df_text[1] !=Prefix_list[15])
        
                          ] 
            
    for i in idx_fail.index:
         for j in Prefix_list:
                # ตรวจดูข้อมูลแล้วเปลี่ยนให้ถูกต้องตามแต่กรณี
                if (df_text[2].loc[i] == j):
                    df_text[0].loc[i] = df_text[0].loc[i]+' '+df_text[1].loc[i] # รวมคอลัมน์ 0, 1
                    df_text[1].loc[i] = j                 # แทนที่คอลัมน์ 1 ด้วยคอลัมน์ 2
                    
                elif df_text[2].loc[i] == 'Countess.':
                    df_text[1].loc[i] = 'the Countess.' # แทนที่ the ด้วย 'the Countess.'
                    
                elif df_text[3].loc[i] == j:
                    df_text[0].loc[i] = df_text[0].loc[i]+' '+df_text[1].loc[i]+' '+ df_text[2].loc[i] # รวมคอลัมน์ 0, 1, 2
                    df_text[1].loc[i] = j          # แทนที่คอลัมน์ 2 ด้วยคอลัมน์ 3
                    
    for i in df_text.index:
        
          for j in Prefix_list:
                

            # กรณีคอลัมน์ 1 = Prefix_list , คอลัมน์ 2 = Prefix_list
                if (df_text[1].loc[i]==j) & (df_text[2].loc[i]==j):
                    df_text[2].loc[i] = ''
                    k = 2
                    for k in range(len(df_text.loc[i])-3):
                        df_text[2].loc[i] += df_text[k+3].loc[i] + ' '
            
            # กรณีคอลัมน์ 1 = Prefix_list , คอลัมน์ 3 = Prefix_list
                elif (df_text[1].loc[i]==j) & (df_text[2].loc[i]!=j) &(df_text[3].loc[i]==j):
                    df_text[2].loc[i] = ''
                    df_text[3].loc[i] = ''
                    k = 2
                    for k in range(len(df_text.loc[i])-2):
                        df_text[2].loc[i] += df_text[k+2].loc[i] + ' '
             # ทั่วไป กรณีคอลัมน์ 1 = Prefix_list  
                elif (df_text[1].loc[i]==j) & (df_text[2].loc[i]!=j) &(df_text[3].loc[i]!=j):
                    k = 2
                    for k in range(len(df_text.loc[i])-3):
                        df_text[2].loc[i] += ' ' + df_text[k+3].loc[i]
          
    # กรณีคำนำหน้าเป็น 'the Countess.'   
    if (df_text[1].loc[i]== 'the Countess.'):
        
        k = 2
        df_text[2].loc[i] = ' '
        for k in range(len(df_text.loc[i])-2):
            df_text[2].loc[i] += ' ' + df_text[k+2].loc[i]    
     
    # เก็บไว้ใน DataFrame พร้อมตั้งชื่อคอลัมน์
    column_features = ['last_name', 'prefix_name', 'name']
    i = 0    
    for feature in column_features: 
        df_text[feature] = df_text[i]
        i += 1
        
    # เลือก features ที่จะนำมาใช้
    cols = ['last_name', 'prefix_name', 'name']
    X_features = df_text[cols]
    
    # นับจำนวน : word_lastName, syllable_lastName, word_name,syllable_name                            
    tfv = TfidfVectorizer() # แบ่งคำ
    count_word_LastName = []
    # สำหรับนับจำนวนคำ word_lastName
    for i in X_features.index:
        
        text = X_features.last_name[i]
        text_list = [text]
        words = tfv.fit_transform(text_list)
        text_cut = tfv.get_feature_names()
        count_word_LastName.append(len(text_cut))
        
    # สำหรับนับจำนวนพยางค์ syllable_Lastname     
    X_features['count_word_LastName'] = count_word_LastName    
    count_syllable_LastName = []
    for i in X_features.index:
        count_syllable_LastName.append(syllables(X_features.last_name[i]))

    X_features['count_syllable_LastName'] = np.array(count_syllable_LastName)

    # สำหรับนับจำนวนคำ word_name
    count_word_Name = []
    for i in X_features.index:
        text = X_features.name[i]
        text_list = [text]
        words = tfv.fit_transform(text_list)
        text_cut = tfv.get_feature_names()
        count_word_Name.append(len(text_cut)) 
    X_features['count_word_Name'] = np.array(count_word_Name)    
    
    # สำหรับนับจำนวนพยางค์ syllable_name
    count_syllable_Name = []
    for i in X_features.index:
        count_syllable_Name.append(syllables(X_features.name[i]))
       
    X_features['count_syllable_Name'] = np.array(count_syllable_Name) 
    
    col_sel = ['prefix_name', 'count_word_LastName', 'count_syllable_LastName', 'count_word_Name', 'count_syllable_Name']
    X_pp = X_features[col_sel]
    
    return  X_pp # ค่า feature สำหรับ preprocessing  

In [238]:
for i in range(2):
    print(i)

0
1


In [239]:
def plus(a,b):
    x = a+b
    return x

def multiple(x):
    x = plus(2,x)*2
    return x

multiple(10)
    

24

In [240]:
# ข้อมูลเราเป็น data Frame
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from  sklearn.feature_extraction.text import TfidfVectorizer

Titanic_data = 'Titanic/train.csv' # เก็บชื่อที่อยู่ไฟล์
df = pd.read_csv(Titanic_data) # โหลดไฟล์
df.head() # preview
X = df.Name

In [272]:
X_pp = preprocessing(X)

In [273]:
X_pp = X_pp[:20]
X_pp['prefix_'] = X_pp[['prefix_name']]

In [274]:
X_pp                    

,prefix_name,count_word_LastName,count_syllable_LastName,count_word_Name,count_syllable_Name,prefix_
0,Mr.,1,1,2,4,Mr.
1,Mrs.,1,2,5,9,Mrs.
2,Miss.,1,3,1,2,Miss.
3,Mrs.,1,3,5,7,Mrs.
4,Mr.,1,2,2,4,Mr.
5,Mr.,1,2,1,2,Mr.
6,Mr.,1,3,1,3,Mr.
7,Master.,1,2,2,4,Master.
8,Mrs.,1,2,4,11,Mrs.
9,Mrs.,1,2,3,8,Mrs.


In [282]:
def label_prediction(X_encode, col):
    #col = 'prefix_name'
    label_encode = []
    columns = ['Mr.', 'Mrs.', 'Miss.', 'Master.', 'Don.', 'Rev.', 'Dr.', 'Mme.', 'Ms.', 'Major.', 'Mlle.', 'Col.', 'Capt.', 'Jonkheer.', 'Sir.', 'Lady.', 'Dona.']
    column_label = [11, 12, 8, 7, 2, 14, 3, 10, 13, 6, 9, 1, 0, 4, 15, 5, 17]
    for i in X_encode.index:
        label_encode.append((X_encode[col].loc[i], i))
        
    for prefix, idx in label_encode:
        
        for i in range(len(columns)):
            
            if prefix == columns[i]:
                X_encode[col].loc[idx] = column_label[i]
    
    return X_encode

In [284]:
prefix_name = 'prefix_name'
X_enc = label_prediction(X_pp, prefix_name)
X_enc

,prefix_name,count_word_LastName,count_syllable_LastName,count_word_Name,count_syllable_Name,prefix_
0,11,1,1,2,4,Mr.
1,12,1,2,5,9,Mrs.
2,8,1,3,1,2,Miss.
3,12,1,3,5,7,Mrs.
4,11,1,2,2,4,Mr.
5,11,1,2,1,2,Mr.
6,11,1,3,1,3,Mr.
7,7,1,2,2,4,Master.
8,12,1,2,4,11,Mrs.
9,12,1,2,3,8,Mrs.


In [ ]:
X_encode

In [ ]:
label_encode

In [ ]:
label_prediction(X_pp, prefix_name)

In [ ]:
result = []
for data, i in label_encode:
    if data == 'Mr.':
        Mr = 1
        result.append(Mr)
        ('0')
    elif data == 'Mrs.':
        Mrs = 2
        result.append(Mrs)
    else:
        none = 'x'
        result.append(none)
result

In [ ]:
column_label = [11, 12, 8, 7, 2, 14, 3, 10, 13, 6, 9, 1, 0, 4, 15, 5, 17]

In [ ]:
len(column_label)